---
layout: post
title: Sprint 4 Review 
description: Review of all sprint 5
comments: true
sticky_rank: 1
---

## Project Cantella

# Purpose Of Programming


- All-in-One Platform: Combines flashcards, quizzes, and the Gemini Integrated Chattutor for academic success.

- Personalized Learning: Adapts to different styles with visual aids, step-by-step help, or quick answers.

- Collaborative Tools: Supports group work and resource sharing for better teamwork.

- Self-Paced Learning: Tracks progress and offers tailored recommendations to keep students on track.

- Accessible Anytime: Provides high-quality learning support anywhere, anytime, for all schedules and goals.

- 24/7 Personal Tutor: Always available to help with academic questions, from quick answers to detailed explanations.

- Personalized Learning: Adapts to your style and level, making it feel tailored to you.

- Time-Saving: No need to search textbooks or websites—answers are instant and reliable.

- Affordable Access: High-quality tutoring made accessible for everyone.

- Future Enhancements: Could include answer ratings, visuals (charts/videos), and adjustable difficulty levels.

- Ultimate Study Tool: Makes learning easy, fun, and super effective, helping students succeed.

## Input/Output Requests

Live Review: I will demonstrate asking "CanTeach" a math question and the response being outputted to the backend

<img src="{{site.baseurl}}/images/cantella_image.png" alt="Cantella Image" style="width:100%; max-width:600px;">

------

Upon running db_init, restore, and backup, the data from the ratings table remains constant.

<img src="{{site.baseurl}}/images/cantella_backend.png" alt="Cantella Image" style="width:100%; max-width:600px;">

In [ ]:
if __name__ == "__main__":
    if not app.debug or os.environ.get("WERKZEUG_RUN_MAIN") == "true":
        with app.app_context():
            db.create_all()  # Ensure tables are created before initialization
            if not User.query.first():  # Initialize only if no users exist
                initUsers()
            if not Flashcard.query.first():  # Initialize flashcards only if none exist
                initFlashcards()
            if not GradeLog.query.first():  # Initialize grade logs only if none exist
                initGradeLog()
            if not Profile.query.first():  # Initialize profiles only if none exist
                initProfiles()
            if not Deck.query.first():  # Initialize decks only if none exist
                initDecks()
    app.run(debug=True, host="0.0.0.0", port="8887")

## List Requests

Use of List, Dictionaries, and Database

Code Description:

- Lists: Used to manage user interests.

- Dictionaries: Used to handle JSON data in API requests and responses.

- Database: Used to store user data, including interests.

## Formatting Response Data (JSON) from API into DOM:

In [ ]:
const editButtons = document.querySelectorAll('.edit-button');

editButtons.forEach(button => {
    button.addEventListener('click', async function() {
        // Get the tutor ID (e.g., a unique identifier for CanTeach sessions)
        const tutorId = this.getAttribute('data-id');

        // Fetch tutor-related data from the API
        try {
            const response = await fetch(`/api/tutors/${tutorId}`);
            if (!response.ok) {
                throw new Error(`Failed to fetch data for tutor ID: ${tutorId}`);
            }

            // Parse the JSON response
            const tutorData = await response.json();

            // Format and insert data into the DOM dynamically
            document.getElementById('tutorName').textContent = tutorData.name || 'Unknown Tutor';
            document.getElementById('tutorSpecialty').textContent = tutorData.specialty || 'No specialty provided';
            document.getElementById('tutorStatus').textContent = tutorData.available ? 'Available' : 'Unavailable';

            // Example: Update a chatbox intro message dynamically
            const chatboxIntro = document.getElementById('chatboxIntro');
            chatboxIntro.textContent = `Welcome to your session with ${tutorData.name}, an expert in ${tutorData.specialty}!`;

            // Set hidden input fields if needed (e.g., for updating tutor settings later)
            document.getElementById('tutorId').value = tutorId;
            document.getElementById('tutorAvailable').value = tutorData.available.toString();

        } catch (error) {
            console.error('Error fetching tutor data:', error);
            alert('An error occurred while loading tutor information.');
        }
    });
});


## Queries from database

In [ ]:
def get(self):
    topic = request.args.get('topic')  # Extract the topic from the query parameter
    if not topic:
        return {'message': 'Topic parameter is required'}, 400

    try:
        tutors = db_session.query(Tutor).filter(Tutor.topics_of_expertise.any(topic)).all()

        if not tutors:
            return {'message': f'No tutors found specializing in {topic}'}, 404

        tutor_list = [
            {
                'id': tutor.id,
                'name': tutor.name,
                'experience': tutor.experience_years,
                'rating': tutor.average_rating
            }
            for tutor in tutors
        ]
        return jsonify(tutor_list)

    except Exception as e:
        return {'message': 'An error occurred while fetching tutors', 'error': str(e)}, 500


## Methods in class to Work with Columns:

In [ ]:
class TopicsAPI:
    class _CRUD(Resource):
        @token_required()
        def post(self):
            """
            Add new topics of expertise to the authenticated tutor.
            """
            current_tutor = g.current_user  # Assume current authenticated tutor is stored here
            body = request.get_json()
            new_topics = body.get('topics')

            if not new_topics:
                return {'message': 'No topics provided'}, 400

            # Add new topics to the tutor's topics_of_expertise column
            current_tutor.topics_of_expertise.extend(new_topics)  # Assuming this is a list relationship
            db_session.commit()

            return jsonify({'message': 'Topics added successfully', 'topics': current_tutor.topics_of_expertise})

        @token_required()
        def get(self):
            """
            Retrieve the topics of expertise for the authenticated tutor.
            """
            current_tutor = g.current_user
            return jsonify({'topics': current_tutor.topics_of_expertise})

        @token_required()
        def put(self):
            """
            Update the topics of expertise for the authenticated tutor.
            """
            current_tutor = g.current_user
            body = request.get_json()
            updated_topics = body.get('topics')

            if not updated_topics:
                return {'message': 'No topics provided'}, 400

            # Replace current topics with new ones
            current_tutor.topics_of_expertise = updated_topics
            db_session.commit()

            return jsonify({'message': 'Topics updated successfully', 'topics': current_tutor.topics_of_expertise})

        @token_required()
        def delete(self):
            """
            Delete specific topics from the authenticated tutor.
            """
            current_tutor = g.current_user
            body = request.get_json()
            topics_to_remove = body.get('topics')

            if not topics_to_remove:
                return {'message': 'No topics provided for deletion'}, 400

            # Remove topics from the list
            current_tutor.topics_of_expertise = [
                topic for topic in current_tutor.topics_of_expertise if topic not in topics_to_remove
            ]
            db_session.commit()

            return jsonify({'message': 'Topics deleted successfully', 'topics': current_tutor.topics_of_expertise})

Explanation: 

- Class Overview: TopicsAPI._CRUD provides CRUD operations for managing a tutor's topics of expertise, including adding, retrieving, updating, and deleting topics.

- POST Method: Accepts a list of new topics from the request body, adds them to the tutor's current list, and commits the changes to the database.

- GET Method: Retrieves and returns the authenticated tutor's current list of expertise topics in JSON format.

- PUT Method: Replaces the tutor's entire list of topics with a new list provided in the request body and saves the changes.

- DELETE Method: Removes specific topics from the tutor's list of expertise based on the request data and updates the database.

## Algorithmic Code Request

## Definition of Code Blocks to Handle a Request

API Class to Perform CRUD Methods:

In [ ]:
class SubjectsAPI:
    class _CRUD(Resource):
        @token_required()
        def post(self):
            """
            Add new subjects to the authenticated tutor.
            """
            current_tutor = g.current_user
            body = request.get_json()
            new_subjects = body.get('subjects')

            if not new_subjects:
                return {'message': 'No subjects provided'}, 400

            # Add new subjects to the tutor's list
            current_tutor.subjects = current_tutor.subjects + new_subjects if current_tutor.subjects else new_subjects
            db_session.commit()

            return jsonify({'message': 'Subjects added successfully', 'subjects': current_tutor.subjects})

        @token_required()
        def put(self):
            """
            Update the subjects of the authenticated tutor.
            """
            current_tutor = g.current_user
            body = request.get_json()
            updated_subjects = body.get('subjects')

            if not updated_subjects:
                return {'message': 'No subjects provided'}, 400

            # Replace the tutor's current subjects with the new list
            current_tutor.subjects = updated_subjects
            db_session.commit()

            return jsonify({'message': 'Subjects updated successfully', 'subjects': current_tutor.subjects})

        @token_required()
        def delete(self):
            """
            Delete a specific subject from the authenticated tutor's expertise.
            """
            current_tutor = g.current_user
            body = request.get_json()

            if not body or 'subject' not in body:
                return {'message': 'No subject provided'}, 400

            subject_to_delete = body.get('subject')

            if subject_to_delete not in current_tutor.subjects:
                return {'message': 'Subject not found'}, 404

            # Remove the subject and update the database
            current_tutor.subjects = [subject for subject in current_tutor.subjects if subject != subject_to_delete]
            db_session.commit()

            return jsonify({'message': 'Subject deleted successfully', 'subjects': current_tutor.subjects})

        @token_required()
        def get(self):
            """
            Retrieve the subjects of expertise for the authenticated tutor.
            """
            current_tutor = g.current_user
            return jsonify({'subjects': current_tutor.subjects})


Explaintion: 

- Class Purpose: SubjectsAPI._CRUD provides endpoints for authenticated tutors to manage their subjects of expertise (CRUD: Create, Read, Update, Delete).

- POST Method: Adds new subjects to the tutor's expertise. If no subjects are provided, it returns an error message.

- PUT Method: Replaces the tutor's existing subjects with a new list provided in the request. Ensures the list is valid before updating.

- DELETE Method: Deletes a specific subject from the tutor's expertise list if it exists. If the subject is not found, it returns an error message.

- GET Method: Retrieves and returns the current list of subjects the tutor specializes in as a JSON response.

## Method/Procedure in Class with Sequencing, Selection, and Iteration 

In [ ]:
@token_required()
def delete(self):
    """
    Remove a specified student from the authenticated tutor's assigned list.
    """
    body = request.get_json()

    # Selection: Check if the required data is provided in the request body
    if not body or 'student_id' not in body:
        return {'message': 'No student ID provided'}, 400

    current_tutor = g.current_user  # Assume the current authenticated tutor
    student_id_to_remove = body.get('student_id')

    # Sequence: Retrieve the list of assigned students
    assigned_students = current_tutor.assigned_students  # Assume this is a list of student IDs

    # Selection: Check if the student ID exists in the tutor's assigned list
    if student_id_to_remove not in assigned_students:
        return {'message': 'Student not found in assigned list'}, 404

    # Iteration: Remove the student by filtering the list
    current_tutor.assigned_students = [
        student_id for student_id in assigned_students if student_id != student_id_to_remove
    ]

    # Sequence: Save changes to the database
    db_session.commit()

    return {
        'message': 'Student removed successfully',
        'assigned_students': current_tutor.assigned_students
    }


Explanation: 

- Purpose: This method removes a specific student from the authenticated tutor's assigned list based on the student_id provided in the request.

- Sequencing: The method fetches the request body, retrieves the tutor's assigned students list, removes the specified student, and commits the updated list to the database in sequential steps.

- Selection: It checks if the student_id is provided in the request body and ensures that the student exists in the tutor’s assigned list before proceeding.

- Iteration: A list comprehension iterates over the assigned_students list to filter out the student with the matching student_id.

- Response: Returns a success message along with the updated list of assigned students if the operation completes successfully; otherwise, returns an appropriate error message.

## Parameters and Return Type

Parameters:

body: JSON object containing the interest to be deleted.

Return Type:

JSON response with a message indicating the result of the operation.


## Call to Algorithm Request

## Definition of Code Block to Make a Request

Frontend Fetch to Endpoint:

In [ ]:
async function deleteSubject(subject) {
    try {
        const response = await fetch(pythonURI + "/api/subjects", {
            ...fetchOptions,
            method: 'DELETE',
            body: JSON.stringify({ subject: subject })
        });

        if (!response.ok) {
            throw new Error('Failed to delete subject');
        }

        // Provide feedback to the user
        showError('Subject deleted successfully', 'green');
        updateTutorInfo(); // Refresh the tutor's information (e.g., subject list)
    } catch (error) {
        console.error('Error deleting subject:', error);
        showError('Error deleting subject');
    }
}

window.deleteSubject = deleteSubject;


Explanation:


- Purpose: The deleteSubject function sends a DELETE request to the /api/subjects endpoint to remove a specific subject from the tutor's list of expertise.

- API Interaction: It includes the subject to be deleted in the request body (formatted as JSON) and uses the fetch API to communicate with the backend.

- Error Handling: If the response is not successful, the function throws an error, logs the issue to the console, and displays an error message to the user.

- Success Feedback: On successful deletion, a success message is displayed to the user (e.g., "Subject deleted successfully"), and the tutor's updated information is fetched using the updateTutorInfo() function.

- Global Functionality: The function is assigned to the window object, making it globally accessible and callable from other parts of the application, such as a "Delete" button in the UI.


## Discuss the Call/Request to the Method with Algorithm

Call/Request:

- The deleteInterest function is responsible for sending a DELETE request to the /api/interests endpoint. It includes the specific interest to be removed as part of the request.

Return/Response:

- The response from the backend is handled by checking the status code and updating the UI accordingly.

## Handling Data and Error Conditions
Normal Conditions:

- The interest is successfully deleted, and the UI is updated to reflect the change.

Error Conditions:

- If the interest is not found or the request fails, an error message is displayed.